In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')
import tokamakTK

from tokamakTK import get_metrics_for_decreasing

import pydotplus
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from sklearn.tree import plot_tree
from sklearn.inspection import permutation_importance
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


pd.set_option('display.max_columns', None)
path = "../../data/"
fig_path = "../../../../LATEX/Latex Images/"

sstyle = 'seaborn-v0_8-poster'
plt.style.use(sstyle)
plt.rc('font',family = 'serif')

In [3]:
# Obtained from Optimization

min_subset_ids = pd.read_csv(path+"R_ids_alpha_0.6357.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# REMOVING SPHERICAL TOKAMAKS
#DB5 = DB5[~DB5.TOK.isin(['MAST', 'NSTX', 'START'])]


# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label"] = 1


print(
    f"{ round( (len(min_subset_ids)/len(DB5))*100     ,2)  }% of the data decreased alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids)/len(DB5))*100 ,2)  }% of the data did not decrease alpha_R"
)

23.45% of the data decreased alpha_R
76.55% of the data did not decrease alpha_R


In [4]:
plasma_characteristics = ["Q95","BEIMHD","PREMAG",
                          "CONFIG","DWDIA","WMHD","TORQ","KAREA", "EPS","MEFF","VOL","LCOULOMB",
                          "IP","RHOSTAR","NUSTAR","BETASTAR"] 
TOK_characteristics = ["TOK","WALMAT","DIVMAT","LIMMAT","AMIN","BT"]
ELM = ["ELMTYPE","ELMFREQ"]
heating = ["PECRH", "PICRH", "ICSCHEME","AUXHEAT","ECHMODE","PELLET"]
impurities = ["EVAP","ZEFF","ZEFFNEO","PRAD","POHM","ENBI","PNBI"]
power = ["PLTH","PFLOSS"]
temperatures = ["TAV","TEV","TIV"]
fast_particles = ["NESOL","WFFORM","WFICFORM","OMEGACYCL","NEL"] 

features = plasma_characteristics + TOK_characteristics + ELM + \
           heating + impurities + power + temperatures + fast_particles

In [5]:
categorical_features = DB5[features].select_dtypes(include=['object']).columns.tolist()

# 3 subsets to study
research_features = ['NEL','TAV','BT','RHOSTAR','NUSTAR','BETASTAR'] + categorical_features
entropy_low_MCL = ['WFFORM', 'RHOSTAR', 'DWDIA', 'BETASTAR', 'POHM', 'NEL', 'NUSTAR', 'PLTH'] + categorical_features
low_MLC = ['BEIMHD', 'BETASTAR', 'BT', 'DWDIA', 'ELMFREQ', 'IP', 'NEL', 'NESOL', 'NUSTAR', 
           'PECRH', 'PFLOSS', 'PICRH', 'POHM', 'PRAD', 'Q95', 'RHOSTAR', 'TIV', 'TORQ', 
           'WFFORM', 'WFICFORM', 'ZEFF', 'ZEFFNEO']
FEATURES = [research_features, entropy_low_MCL, low_MLC]

In [ ]:
def split_clean_data(features, db5):
    DB5 = tokamakTK.clean_categorical_data(db5)
    # Needed to respectively clean each dtype
    num_features = DB5[features].select_dtypes(include=['int', 'float']).columns.tolist()
    cat_features = DB5[features].select_dtypes(include=['object']).columns.tolist()
    data_num = DB5[num_features+["TOK","DATE"]]
    data_cat = DB5[cat_features]
    data_num = tokamakTK.clean_numerical_data(data_num)
    data_ = pd.concat([data_num,
                      (pd.concat([
                           DB5[["label"]], 
                           tokamakTK.encode_categorical_ohe(data_cat)
                          ], axis=1)
                      )],
                      axis=1)
    X = data_.drop("label", axis=1)
    y = data_["label"]
    # split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=71, stratify=y)
    return X_train, X_test, y_train, y_test

def define_model(X_train, y_train, model_="RF"):
    gp = GaussianProcessClassifier(random_state=71, n_jobs=-1)
    if model_ == "RF":
        model = RandomForestClassifier(min_samples_split=2,  
                                min_samples_leaf=1,
                                min_impurity_decrease=0.0007,
                                n_estimators = 90,
                                max_depth = 23,
                                criterion='entropy',
                                max_features=None,
                                n_jobs=-1,
                                random_state=71
                               )
    else:
        model = GaussianProcessClassifier(random_state=71, n_jobs=-1)
    model.fit(X_train, y_train)
    return model

def get_predictions(model, X_test):
    y_pred = model.predict_proba(X_test)
    return y_pred

def get_PRCs(results, label_pos=["lower left"]*3, saveFig=False, file_name=""):
    LABELS = ["Research", "Entropy with low MCL", "Low MCL"]
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(20, 5.7))
    for i, result in enumerate(results):
        axs[i].plot(result[1], result[0], marker='.', label=LABELS[i], c="#A84547")
        axs[i].grid(alpha=0.5)
        axs[i].legend(loc=label_pos[i])
    axs[0].set_ylabel("Precision\n", fontsie=21)
    axs[1].set_xlabel("\nRecall", fontsie=21)
    if saveFig:
        plt.savefig(fig_path+file_name+".pdf", format="pdf", dpi=800, bbox_inches='tight');
    plt.show()
    
def get_importance(model, X_training, X_test, y_test, n_permutations=200, metric="f1_macro"):
    """
    Returns pd.DataFrame with Ordered Feature Importance based on Permutations
    """
    Importance = pd.DataFrame(np.zeros((len(X_training.columns), 2)), 
                              index=X_training.columns, columns=["mean", "std"])
    permutation = permutation_importance(model, X_test, y_test, n_repeats=n_permutations, 
                                         n_jobs=-1,
                                         scoring=metric,
                                         random_state=71)
    for i in permutation.importances_mean.argsort()[::-1]:
        if permutation.importances_mean[i] - 2 * permutation.importances_std[i] > 0:
            Importance.loc[X_training.columns[i], "mean"] = np.round(permutation.importances_mean[i],4)
            Importance.loc[X_training.columns[i], "std"] = np.round(permutation.importances_std[i],4)
    Importance.sort_values("mean", ascending=False, inplace=True)
    return Importance

In [7]:
SPLIT_DATA = [split_clean_data(f, db5=DB5) for f in FEATURES]
GaussianP  = [define_model(split_data[0], split_data[2], model_="GP") for split_data in SPLIT_DATA]
RdForests  = [define_model(split_data[0], split_data[2], model_="RF") for split_data in SPLIT_DATA]

In [8]:
GaussianP_preds = [get_predictions(GaussianP[i], SPLIT_DATA[i][1]) for i in range(3)]
RdForests_preds = [get_predictions(RdForests[i], SPLIT_DATA[i][1]) for i in range(3)]

In [9]:
GP_PRCs = [precision_recall_curve(SPLIT_DATA[i][3], GaussianP_preds[i][:,1]) for i in range(3)]
RF_PRCs = [precision_recall_curve(SPLIT_DATA[i][3], RdForests_preds[i][:,1]) for i in range(3)]

In [ ]:
get_PRCs(GP_PRCs, saveFig=True, file_name="GP_PRCs")

In [ ]:
get_PRCs(RF_PRCs, saveFig=True, file_name="RF_PRCs")

In [25]:
METRICS = pd.DataFrame(np.zeros((6,4)), columns=["Accuracy", "Precision", "Recall", "F1score"],
            index=[f"GP{i}" for i in range(3)]+[f"RF{i}" for i in range(3)])

for i in range(3):
    gp_metrics = get_metrics_for_decreasing(GaussianP[i], X_test=SPLIT_DATA[i][1], y_test=SPLIT_DATA[i][3])
    rf_metrics = get_metrics_for_decreasing(RdForests[i], X_test=SPLIT_DATA[i][1], y_test=SPLIT_DATA[i][3])
    
    METRICS.loc[f"GP{i}", "Accuracy"] = gp_metrics[0]
    METRICS.loc[f"GP{i}", "Precision"] = gp_metrics[1]
    METRICS.loc[f"GP{i}", "Recall"] = gp_metrics[2]
    METRICS.loc[f"GP{i}", "F1score"] = gp_metrics[3]
    
    METRICS.loc[f"RF{i}", "Accuracy"] = rf_metrics[0]
    METRICS.loc[f"RF{i}", "Precision"] = rf_metrics[1]
    METRICS.loc[f"RF{i}", "Recall"] = rf_metrics[2]
    METRICS.loc[f"RF{i}", "F1score"] = rf_metrics[3]
    
METRICS.index = ["GP_research", "GP_entropyLM", "GP_lowMCL", "RF_research", "RF_entropyLM", "RF_lowMCL"]
METRICS.sort_values("F1score", ascending=False, inplace=True)

In [ ]:
GP_Imporance = [get_importance(GaussianP[i], 
                               SPLIT_DATA[i][0], SPLIT_DATA[i][1], SPLIT_DATA[i][3]) for i in range(3)]
RF_Imporance = [get_importance(RdForests[i], 
                               SPLIT_DATA[i][0], SPLIT_DATA[i][1], SPLIT_DATA[i][3]) for i in range(3)]

In [ ]:
import chime
chime.success()